# Roman Emperors
## 27 BCE - 375 CE


## Contents
* <a href='#Introduction'>Introduction</a>
* <a href="#Data-Loading">Data Loading</a>
* <a href="#Data-Transformation">Data Transformation</a>
* <a href="#Creating-the-Vega-Specification">Creating the Vega Specification</a>
* <a href="#Generating-the-SVG">Generating the SVG</a>
* <a href="#Cleaning-the-SVG">Cleaning the SVG</a>
* <a href="#Previewing-the-SVG">Previewing the SVG</a>

### Introduction
Here I juxtapose the reigns of a subset of Roman emperors against the (literal) arc of their lives. 

Some things to note:

* The analysis here is superficial and I have not performed the kind of thorough consultation of primary sources necessary to formulate solid conclusions regarding manner of death. I am not a historian nor otherwise trained in historical methods. Dates in the original dataset are also largely estimates, and so I therefore gladly invite corrections to these data where necessary.


* Dates here are sourced from a dataset compiled by others [here](https://github.com/zonination/emperors). Note in particular that the absolute values of BCE dates are decreased by 1 to conform to the [ISO 8601](https://en.wikipedia.org/wiki/ISO_8601) standard.

  - Thanks to [zonination](https://github.com/zonination) for creating this dataset.
  - Note: I did manually make some corrections to names in the final visualization.

* While I did rely on the aforementioned dataset for dates, I chose to review a different source for information on the deaths of emperors. I principally reviewed *[De Imperatoribus Romanis](https://isaw.nyu.edu/publications/awol-index/html/www.luc.edu/roman-emperors.html)*, a peer-reviewed encyclopedia of Roman rulers which I judged to be suitable for this project and which I noted as being used for a statistical analysis in a [recent scholarly publication](https://www.nature.com/articles/s41599-019-0366-y). It is recognized, however, that the writings there are generally decades old and may not reflect the most current scholarly consensus. Again, I invite corrections.

    - Thanks to Christian Körner, Christopher J. Fuhrmann, David Wend, David Woods, Dennis Quinn, Garrett G. Fagan, Geoffrey Nathan, Hans A. Pohlsander, Herbert W. Benario, John Donahue, Michael DiMaio, Jr., Michael L. Meckler, Noel Lenski, Phoebe B. Peacock, R. Scott Moore, Ralph W. Mathisen, Richard D. Weigel, Robert Frakes, Robin McMahon, Thomas Banchich, Walter E. Roberts, and William Leadbetter for their contributions to *De Imperatoribus Romanis* which I reviewed in annotating probable manner of death. 


* I mapped the descriptions of emperors deaths in *De Imperatoribus Romanis* to the same death classification scheme used in the [original dataset's repository](https://github.com/zonination/emperors), as it seemed to reasonably capture the meaningful context of all emperors' deaths in the broad manner necessary for this visualization.

* Finally, thanks to David Bacci for showcasing the [Coronation Arc chart](https://github.com/PBI-David/Deneb-Showcase/blob/main/Coronation%20Arc%20Chart/Thumbnail.png) marking the coronation of Charles III of the United Kingdom. I was immediately struck by the elegance of that visualization and I am grateful for the availability of his Vega specifications which inspired this project. 


In [1]:
# For basic data handling
import pandas as pd

# For regex-based matching of atypical date strings
import re

# For handling of propleptic Gregorian dates
import astropy
from astropy.time import Time

# For making a proper JSON dump of our Vega specification
import json

# For calling vega-CLI and svgo from Python
import subprocess

# For previewing the SVG file we generate here
from IPython.display import SVG, display

### Data Loading

In [2]:
# Reading our data source in directly from its repo
df_original = pd.read_csv(r'https://github.com/zonination/emperors/raw/refs/heads/master/emperors.csv', encoding = 'CP1252')
df_original.shape

(68, 16)

In [3]:
# Pretty-prints the first five rows of the DataFrame
df_original.head()

,index,name,name.full,birth,death,birth.cty,birth.prv,rise,reign.start,reign.end,cause,killer,dynasty,era,notes,verif.who
0,1,Augustus,IMPERATOR CAESAR DIVI FILIVS AVGVSTVS,0062-09-23,0014-08-19,Rome,Italia,Birthright,0026-01-16,0014-08-19,Assassination,Wife,Julio-Claudian,Principate,"birth, reign.start are BCE. Assign negative fo...",Reddit user zonination
1,2,Tiberius,TIBERIVS CAESAR DIVI AVGVSTI FILIVS AVGVSTVS,0041-11-16,0037-03-16,Rome,Italia,Birthright,0014-09-18,0037-03-16,Assassination,Other Emperor,Julio-Claudian,Principate,birth is BCE. Assign negative for correct ISO ...,Reddit user zonination
2,3,Caligula,GAIVS IVLIVS CAESAR AVGVSTVS GERMANICVS,0012-08-31,0041-01-24,Antitum,Italia,Birthright,0037-03-18,0041-01-24,Assassination,Senate,Julio-Claudian,Principate,assassination may have only involved the Praet...,Reddit user zonination
3,4,Claudius,TIBERIVS CLAVDIVS CAESAR AVGVSTVS GERMANICVS,0009-08-01,0054-10-13,Lugdunum,Gallia Lugdunensis,Birthright,0041-01-25,0054-10-13,Assassination,Wife,Julio-Claudian,Principate,birth is BCE. Assign negative for correct ISO ...,Reddit user zonination
4,5,Nero,NERO CLAVDIVS CAESAR AVGVSTVS GERMANICVS,0037-12-15,0068-06-09,Antitum,Italia,Birthright,0054-10-13,0068-06-09,Suicide,Senate,Julio-Claudian,Principate,NaN,Reddit user zonination


In [4]:
# Check data types  
df_original.dtypes  

index           int64
name           object
name.full      object
birth          object
death          object
birth.cty      object
birth.prv      object
rise           object
reign.start    object
reign.end      object
cause          object
killer         object
dynasty        object
era            object
notes          object
verif.who      object
dtype: object

In [5]:
# Check for missing values  
df_original.isnull().sum()

index           0
name            0
name.full       0
birth           5
death           0
birth.cty      17
birth.prv       0
rise            0
reign.start     0
reign.end       0
cause           0
killer          0
dynasty         0
era             0
notes          22
verif.who      57
dtype: int64

Several individuals appear to have null birth dates. Let's see who they are!

In [6]:
df_original[df_original['birth'].isnull()]

,index,name,name.full,birth,death,birth.cty,birth.prv,rise,reign.start,reign.end,cause,killer,dynasty,era,notes,verif.who
43,44,Florian,CAESAR MARCVS ANNIVS FLORIANVS AVGVSTVS,NaN,0276-09-15,NaN,Unknown,Birthright,0276-06-15,0276-09-15,Assassination,Own Army,Gordian,Principate,"death, reign.end, reign.start are estimates. B...",NaN
46,47,Numerian,CAESAR MARCVS AVRELIVS NVMERIVS NVMERIANVS AVG...,NaN,0284-11-20,NaN,Unknown,Birthright,0283-08-01,0284-11-20,Unknown,Unknown,Gordian,Principate,"death, reign.end, reign.start are estimates. B...",NaN
47,48,Carinus,CAESAR MARCVS AVRELIVS CARINVS AVGVSTVS,NaN,0285-08-01,NaN,Unknown,Birthright,0283-08-01,0285-08-01,Died in Battle,Opposing Army,Gordian,Principate,"death, reign.end, reign.start are estimates. B...",NaN
52,53,Severus II,FLAVIVS VALERIVS SEVERVS AVGVSTVS,NaN,0307-09-16,NaN,Unknown,Appointment by Emperor,0305-05-01,0307-09-16,Assassination,Other Emperor,Constantinian,Dominate,NaN,NaN
60,61,Vetranio,CAESAR FLAVIVS VETRANIO AVGVSTVS,NaN,0356-01-01,NaN,Moesia,Seized Power,0350-03-01,0350-12-25,Unknown,Unknown,Constantinian,Dominate,death is only estimate,NaN


In [7]:
# Check unique values in categorical columns  
df_original.nunique()

index          68
name           68
name.full      67
birth          60
death          66
birth.cty      30
birth.prv      18
rise            8
reign.start    57
reign.end      65
cause           7
killer         14
dynasty         8
era             2
notes          23
verif.who       2
dtype: int64

**Out of curiosity, let's see which emperors supposedly shared a full name.**

In [8]:
# Check for duplicate values for `name.full`
duplicates = df_original[df_original.duplicated(subset = 'name.full', keep = False) == True]
duplicates.head()

,index,name,name.full,birth,death,birth.cty,birth.prv,rise,reign.start,reign.end,cause,killer,dynasty,era,notes,verif.who
8,9,Vespasian,TITVS FLAVIVS CAESAR VESPASIANVS AVGVSTVS,0009-11-17,0079-06-24,Falacrine,Italia,Seized Power,0069-12-21,0079-06-24,Natural Causes,Disease,Flavian,Principate,NaN,NaN
9,10,Titus,TITVS FLAVIVS CAESAR VESPASIANVS AVGVSTVS,0039-12-30,0081-09-13,Rome,Italia,Birthright,0079-06-24,0081-09-13,Natural Causes,Disease,Flavian,Principate,NaN,NaN


### Data Transformation

#### Things to do:
1. Select only the columns we need.
2. Filter-out individuals with null birth dates.
3. Assign negative values to BCE dates:
    - [ ] Augustus's birth and reign start
    - [ ] Tiberius's birth
    - [ ] Claudius's birth
4. Calculate the age of each individual at the beginning of their reign.
5. Calculate the age of each individual at the end of their reign.
6. Calculate the age of each individual at their death.

#### Select needed columns.

In [9]:
minified_df = df_original[['index', 'name', 'name.full', 'birth', 'death', 'reign.start', 'reign.end', 'cause']]

#### Filter-out individuals with null birth dates

In [10]:
df_with_birth = minified_df[minified_df ['birth'].notnull()].copy()
df_with_birth.shape

(63, 8)

In [11]:
df_without_birth = minified_df[minified_df ['birth'].isnull()].copy()
df_without_birth.shape

(5, 8)

#### Assign negative values to BCE dates

In [12]:
# Augustus
df_with_birth.loc[df_with_birth['name'] == 'Augustus', 'birth'] = '-' + df_with_birth['birth'].astype(str)
df_with_birth.loc[df_with_birth['name'] == 'Augustus', 'reign.start'] = '-' + df_with_birth['reign.start'].astype(str)

# Tiberius
df_with_birth.loc[df_with_birth['name'] == 'Tiberius', 'birth'] = '-' + df_with_birth['birth'].astype(str)

# Claudius
df_with_birth.loc[df_with_birth['name'] == 'Claudius', 'birth'] = '-' + df_with_birth['birth'].astype(str)

# Routine check for accidents
df_with_birth.shape

(63, 8)

In [13]:
# Another quick check for any unintended mishaps
df_with_birth.head()

,index,name,name.full,birth,death,reign.start,reign.end,cause
0,1,Augustus,IMPERATOR CAESAR DIVI FILIVS AVGVSTVS,-0062-09-23,0014-08-19,-0026-01-16,0014-08-19,Assassination
1,2,Tiberius,TIBERIVS CAESAR DIVI AVGVSTI FILIVS AVGVSTVS,-0041-11-16,0037-03-16,0014-09-18,0037-03-16,Assassination
2,3,Caligula,GAIVS IVLIVS CAESAR AVGVSTVS GERMANICVS,0012-08-31,0041-01-24,0037-03-18,0041-01-24,Assassination
3,4,Claudius,TIBERIVS CLAVDIVS CAESAR AVGVSTVS GERMANICVS,-0009-08-01,0054-10-13,0041-01-25,0054-10-13,Assassination
4,5,Nero,NERO CLAVDIVS CAESAR AVGVSTVS GERMANICVS,0037-12-15,0068-06-09,0054-10-13,0068-06-09,Suicide


#### Helper functions
Here we'll create a couple of functions to help us parse the date strings from above and do the basic time calculations. Given that it is unrealistic to reliably determine precise ages for even notable figures in antiquity, I round the final values to integers, as further precision is simply not meaningful here. 

In [14]:
def parse_bce_date_astropy(date_str):
    """
    Parse a date string in the format 'YYYY-MM-DD',
    where negative years represent BCE and return an Astropy Time object in proleptic Gregorian.
    
    In proleptic Gregorian:
      1 BCE = year 0
      2 BCE = year -1
      53 BCE = year -52
    """
    match = re.match(r'^(-?\d+)-(\d{2})-(\d{2})$', date_str.strip())
    
    if not match:
        raise ValueError(f"Invalid date format: {date_str}")

    year_str, month_str, day_str = match.groups()
    year = int(year_str)
    month = int(month_str)
    day = int(day_str)

    # Convert BCE => “proleptic negative year”
    # e.g., -0053 => year=-53 => proleptic year=-52 for 53 BCE
    #if year < 0:  <= Unnecessary in this case as the maintainer of repository from which the dataset originates already modified BCE dates in this way
        #year += 1

    # Basic validation
    if not (1 <= month <= 12):
        raise ValueError(f"Invalid month: {month}")
    if not (1 <= day <= 31):
        raise ValueError(f"Invalid day: {day}")

    # Create Astropy Time in proleptic Gregorian
    result = Time(val=dict(year=year, month=month, day=day),
             format='ymdhms', scale='utc')
    return result


def calculate_duration(initial_str, terminal_str):
    """
    Given two date strings (YYYY-MM-DD), calculate the time difference in years (float) using Astropy.
    """
    # Parse each date
    birth_time = parse_bce_date_astropy(initial_str)
    death_time = parse_bce_date_astropy(terminal_str)

    # Subtract the two Time objects
    # result is an Astropy TimeDelta
    delta = death_time - birth_time

    # Convert difference to days
    diff_days = delta.to_value('jd')  # JD = days

    # Convert to years (approx) using a standard tropical year length
    # (You can decide if 365.2425 or 365.25 is appropriate.)
    diff_years = diff_days / 365.2425

    # Round to 1 decimal place
    result = round(diff_years)
    
    return result
    

#### Calculating ages at beginning of reign, end of reign, and death

In [15]:
df = df_with_birth.copy()
df['age_at_reign_start'] = df.apply(lambda x: calculate_duration(x['birth'], x['reign.start']), axis = 1)
df['age_at_reign_end'] = df.apply(lambda x: calculate_duration(x['birth'], x['reign.end']), axis = 1)
df['age_at_death'] = df.apply(lambda x: calculate_duration(x['birth'], x['death']), axis = 1)

C:\Users\admin\emperors\Lib\site-packages\erfa\core.py:133: ErfaWarning: ERFA function "dtf2d" yielded 1 of "dubious year (Note 6)"
  warn(f'ERFA function "{func_name}" yielded {wmsg}', ErfaWarning)
C:\Users\admin\emperors\Lib\site-packages\erfa\core.py:133: ErfaWarning: ERFA function "utctai" yielded 1 of "dubious year (Note 3)"
  warn(f'ERFA function "{func_name}" yielded {wmsg}', ErfaWarning)
C:\Users\admin\emperors\Lib\site-packages\erfa\core.py:133: ErfaWarning: ERFA function "dtf2d" yielded 1 of "dubious year (Note 6)"
  warn(f'ERFA function "{func_name}" yielded {wmsg}', ErfaWarning)
C:\Users\admin\emperors\Lib\site-packages\erfa\core.py:133: ErfaWarning: ERFA function "dtf2d" yielded 1 of "dubious year (Note 6)"
  warn(f'ERFA function "{func_name}" yielded {wmsg}', ErfaWarning)
C:\Users\admin\emperors\Lib\site-packages\erfa\core.py:133: ErfaWarning: ERFA function "utctai" yielded 1 of "dubious year (Note 3)"
  warn(f'ERFA function "{func_name}" yielded {wmsg}', ErfaWarning)
C:\

**Note**: The warnings above are related to the years being processed predating the use of the UTC time scale. As discussed [here](https://github.com/astropy/astropy/issues/5809), issues can arise from performing calculations involving dates too far in the future or past which are of critical importance for astronomical applications, but seem vanishingly unlikely to meaningfully affect the very coarse calculation of time that we are performing here (Especially considering the errors that have already arisen from the uncertainty of the data we already have!).

In [16]:
# Prettyprints first five rows of the DataFrame
df.head()

,index,name,name.full,birth,death,reign.start,reign.end,cause,age_at_reign_start,age_at_reign_end,age_at_death
0,1,Augustus,IMPERATOR CAESAR DIVI FILIVS AVGVSTVS,-0062-09-23,0014-08-19,-0026-01-16,0014-08-19,Assassination,35,76,76
1,2,Tiberius,TIBERIVS CAESAR DIVI AVGVSTI FILIVS AVGVSTVS,-0041-11-16,0037-03-16,0014-09-18,0037-03-16,Assassination,55,77,77
2,3,Caligula,GAIVS IVLIVS CAESAR AVGVSTVS GERMANICVS,0012-08-31,0041-01-24,0037-03-18,0041-01-24,Assassination,25,28,28
3,4,Claudius,TIBERIVS CLAVDIVS CAESAR AVGVSTVS GERMANICVS,-0009-08-01,0054-10-13,0041-01-25,0054-10-13,Assassination,49,63,63
4,5,Nero,NERO CLAVDIVS CAESAR AVGVSTVS GERMANICVS,0037-12-15,0068-06-09,0054-10-13,0068-06-09,Suicide,17,30,30


### Creating the Vega Specification

#### Composing a JSON array containing the prepared data

In [17]:
# Selects only the columns we need.
    # Keeping 'index' gives us an easy way of sorting data for the visual
df = df[['index', 'name', 'age_at_reign_start', 'age_at_reign_end', 'age_at_death']]

In [18]:
# Prettyprints first five rows of the DataFrame
df.head()

,index,name,age_at_reign_start,age_at_reign_end,age_at_death
0,1,Augustus,35,76,76
1,2,Tiberius,55,77,77
2,3,Caligula,25,28,28
3,4,Claudius,49,63,63
4,5,Nero,17,30,30


In [19]:
# Transforms the DataFrame to a JSON array
data_json = json.dumps(df.to_dict(orient="records"), indent = 4)

#### Completing the full Vega specification.

Below we have a template Vega specification into which we will insert our data string. I adapted this specification from the *Coronation Arc Chart* specification listed in the *[Deneb-Showcase](https://github.com/PBI-David/Deneb-Showcase?tab=readme-ov-file)* GitHub repository maintained by the generous David Bacci.

In [20]:
import json
import subprocess

# 1) First piece of our Vega specification
vega_preamble = r"""{
  "$schema": "https://vega.github.io/schema/vega/v5.json",
  "aria": false,
  "width": 600,
  "height": 1200,
  "background": "#66023C",
  "padding": {
    "bottom": 20,
    "left": 80,
    "right": 20,
    "top": 20
  },
  "autosize": "fit",
  "signals": [
    {
      "name": "facetWidth",
      "value": 40
    },
    {
      "name": "facetHeight",
      "value": 40
    },
    {
      "name": "legendX",
      "update": "0"
    },
    {
        "name": "legendY",
        "update": "800"
    },
    {
        "name": "keyTitleX",
        "update": "220"
    },
    {
        "name": "keyTitleY",
        "update": "legendY + 10"
    },
    {
        "name": "keyX",
        "update": "220"
    },
    {
        "name": "keyY",
        "update": "840"
    },
    {
      "name": "lifeCircleColour",
      "value": "silver"
    },
    {
      "name": "legendLabelColour",
      "value": "silver"
    },
    {
      "name": "monarchLabelColour",
      "value": "white"
    }
  ],
  "data": [
    {
      "name": "dataset",
      "values":
"""

# The second piece of our specification, comprising our data string. We'll wrap this in brackets [] to form a JSON array, and add a trailing comma after that array.
vega_data = f"""{data_json},"""

# 3) This remainder closes out the JSON.
vega_remainder = r"""
      "transform": [
        {
          "type": "collect",
          "sort": {
            "field": "index",
            "order": "ascending"
          }
        }
      ]
    }
  ],
  "legends": [
    {
      "fill": "color",
      "orient": "none",
      "legendY": {"signal": "legendY"},
      "legendX": {"signal": "legendX"},
      "padding": 0,
      "columns": 2,
      "offset": 80,
      "rowPadding": 5,
      "columnPadding": 30
    }
  ],
  "scales": [
    {
      "name": "color",
      "type": "ordinal",
      "range": ["#FBAE17"]
    }
  ],
  "marks": [
    {
      "type": "group",
      "style": "cell",
      "layout": {
        "columns": 8,
        "padding": {"row": 50, "column": 30},
        "align": "all",
        "bounds": "flush",
        "center": true
      },
      "marks": [
        {
          "type": "group",
          "style": "cell",
          "name": "monarchs",
          "from": {
            "facet": {
              "data": "dataset",
              "name": "facet",
              "groupby": ["name"],
              "sort": {"field": "age_at_reign_start", "order": "ascending"}
            }
          },
          "title": {
            "text": {"signal": "parent.name"},
            "color": {"signal": "monarchLabelColour"},
            "fontSize": 10,
            "fontWeight": "400",
            "dy": 4,
            "orient": "bottom",
            "lineHeight": 0,
            "align": "center",
            "anchor": "middle"
          },
          "encode": {
            "update": {
              "width": {"signal": "facetWidth"},
              "height": {"signal": "facetHeight"}
            }
          },
          "marks": [
            {
              "type": "arc",
              "name": "life",
              "from": {"data": "facet"},
              "encode": {
                "update": {
                  "fill": {"signal": "lifeCircleColour"},
                  "x": {"signal": "item.mark.group.width/2"},
                  "y": {"signal": "item.mark.group.height/2"},
                  "startAngle": {"value": 0},
                  "endAngle": {"signal": "(2*PI/100)*datum[\"age_at_death\"]"},
                  "innerRadius": {"signal": "12"},
                  "outerRadius": {"signal": "13"},
                  "tooltip": {"signal": "item"}
                }
              }
            },
            {
              "type": "arc",
              "name": "reign",
              "from": {"data": "facet"},
              "encode": {
                "update": {
                  "fill": {"value": "#FBAE17"},
                  "x": {"signal": "item.mark.group.width/2"},
                  "y": {"signal": "item.mark.group.height/2"},
                  "startAngle": {"signal": "(2*PI/100)*datum[\"age_at_reign_start\"]"},
                  "endAngle": {
                    "signal": "if(((2*PI/100)*datum[\"age_at_reign_end\"] - (2*PI/100)*datum[\"age_at_reign_start\"]) < 0.03, (2*PI/100)*datum[\"age_at_reign_start\"] + 0.03, (2*PI/100)*datum[\"age_at_reign_end\"])"
                  },
                  "innerRadius": {"signal": "10"},
                  "outerRadius": {"signal": "15"},
                  "tooltip": {"signal": "item"}
                }
              }
            }
          ]
        }
      ]
    },
    {
      "type": "text",
      "name": "keyTitle",
      "encode": {
        "update": {
          "fill": {"signal": "legendLabelColour"},
          "text": {"value": "Life & Reign"},
          "fontWeight": {"value": "bold"},
          "fontSize": {"value": 10},
          "x": {"signal": "keyTitleX"},
          "y": {"signal": "keyTitleY"}
        }
      }
    },
    {
      "type": "arc",
      "name": "key",
      "data": [{}],
      "encode": {
        "update": {
          "fill": {"signal": "lifeCircleColour"},
          "x": {"signal": "keyX"},
          "y": {"signal": "keyY"},
          "startAngle": {"value": 0},
          "endAngle": {"signal": "(2*PI/100)*50"},
          "innerRadius": {"signal": "12"},
          "outerRadius": {"signal": "13"}
        }
      }
    },
    {
      "type": "arc",
      "name": "key2",
      "data": [{}],
      "encode": {
        "update": {
          "fill": {"signal": "lifeCircleColour"},
          "x": {"signal": "keyX"},
          "y": {"signal": "keyY"},
          "startAngle": {"signal": "(2*PI/100)*25"},
          "endAngle": {"signal": "(2*PI/100)*35"},
          "innerRadius": {"signal": "10"},
          "outerRadius": {"signal": "15"}
        }
      }
    },
    {
      "type": "text",
      "name": "birth",
      "from": {"data": "key2"},
      "encode": {
        "update": {
          "fill": {"signal": "legendLabelColour"},
          "text": {"value": "birth"},
          "fontSize": {"value": 10},
          "align": {"value": "center"},
          "x": {"signal": "datum.x"},
          "y": {"signal": "datum.y - datum.outerRadius - 2"}
        }
      }
    },
    {
      "type": "text",
      "name": "death",
      "from": {"data": "key2"},
      "encode": {
        "update": {
          "fill": {"signal": "legendLabelColour"},
          "text": {"value": "death"},
          "fontSize": {"value": 10},
          "align": {"value": "right"},
          "x": {"signal": "datum.x - datum.outerRadius"},
          "y": {"signal": "datum.y"}
        }
      }
    },
    {
      "type": "text",
      "name": "reign",
      "from": {"data": "key2"},
      "encode": {
        "update": {
          "fill": {"signal": "legendLabelColour"},
          "text": {"value": "reign"},
          "fontSize": {"value": 10},
          "align": {"value": "left"},
          "x": {"signal": "datum.x + 17"},
          "y": {"signal": "datum.y + 20"}
        }
      }
    },
    {
      "type": "arc",
      "name": "key3",
      "data": [{}],
      "encode": {
        "update": {
          "fill": {"signal": "lifeCircleColour"},
          "x": {"signal": "keyX + 70"},
          "y": {"signal": "keyY"},
          "startAngle": {"value": 0},
          "endAngle": {"signal": "2*PI"},
          "innerRadius": {"signal": "12"},
          "outerRadius": {"signal": "13"}
        }
      }
    },
    {
      "type": "text",
      "name": "life",
      "from": {"data": "key3"},
      "encode": {
        "update": {
          "fill": {"signal": "legendLabelColour"},
          "text": {"value": "100 yrs"},
          "fontSize": {"value": 6},
          "align": {"value": "center"},
          "baseline": {"value": "middle"},
          "x": {"signal": "datum.x"},
          "y": {"signal": "datum.y"}
        }
      }
    }
  ],
  "config": {
    "view": {"stroke": "red"},
    "style": {
      "col": {"fontSize": 10},
      "cell": {
        "strokeWidth": {"signal": "0"}
      }
    }
  }
}"""

# Combine everything into one final JSON string
final_spec_str = vega_preamble + vega_data + vega_remainder
with open('final_spec_str.txt', 'w') as f:
    f.write(final_spec_str)
    
# Writes the Vega specification to a JSON file
vega_dict = json.loads(final_spec_str)  # now a Python dict
with open("vega.json", "w") as f:
    json.dump(vega_dict, f, indent=4)

### Generating the SVG

I happen to be working on Windows on this particular PC. On Windows, I did the following to make this work: 
- [X] I first installed the `Desktop Development With C++` workload from [Visual Studio 2022](https://visualstudio.microsoft.com/downloads/?q=build+tools).
- [X] Then I installed [Node.js v20.18](https://nodejs.org/en/download). 
- [X] Then I installed the Vega command line interface via `npm install --global vega-cli`

In [21]:
# Writes an SVG Vega visualization to the current working directory/
def vega_to_svg(vega_spec, output_svg_path):
    """
    vega_spec: path to the JSON Vega specification
    output_svg_path: path to output the SVG
    """
    
    # 2. Call vega CLI to compile spec to SVG
    subprocess.run([r"C:\Users\admin\AppData\Roaming\npm\vg2svg.cmd", vega_spec, "output.svg"], check=True)
    print(f"SVG saved to {output_svg_path}")

vega_to_svg("vega.json", "output.svg")

SVG saved to output.svg


### Cleaning the SVG

While the SVG was generated successfully and viewable in my default photo viewer, it was not importable into Adobe Illustrator without further processing; I discovered some [aria](https://en.wikipedia.org/wiki/WAI-ARIA)-related validation errors despite setting `"aria": false` in the specification; I therefore ran it through SVGO to remove problematic parts of the file. You can install svgo via the node package manager using `npm install -g svgo`.

In [22]:
# Invokes svgo to clean the SVG file so we can import it into our vector illustration tool of choice. 
def clean_svg(svg_path):
    """
    svg_path: path to the SVG to be cleaned
    """
    
    # 2. Call vega CLI to compile spec to SVG
    subprocess.run([r"C:\Users\admin\AppData\Roaming\npm\svgo.cmd", svg_path], check=True)
    print("SVG cleaned.")

clean_svg("output.svg")

SVG cleaned.


### Previewing the SVG

Below we can see what our cleaned SVG looks like. From here you can import this SVG into your illustration tool of choice and edit all the different components to your liking.

In [23]:
# Load the SVG file from disk
with open("output.svg", "r", encoding="utf-8") as file:
    svg_data = file.read()

# Display inline in the notebook
display(SVG(svg_data))